# html 2021 final
https://www.kaggle.com/c/html2021final/overview

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# change string to value

In [ ]:
# read CSV
demographics = pd.read_csv('/content/drive/MyDrive/HTML final project/new_demographics.csv')
satisfaction = pd.read_csv('/content/drive/MyDrive/HTML final project/satisfaction.csv')
services = pd.read_csv('/content/drive/MyDrive/HTML final project/new_services.csv')
status = pd.read_csv('/content/drive/MyDrive/HTML final project/status.csv')
location = pd.read_csv('/content/drive/MyDrive/HTML final project/location_with_population_and_fill_in_nan.csv')

Test_IDs = pd.read_csv('/content/drive/MyDrive/HTML final project/Test_IDs.csv')
Train_IDs = pd.read_csv('/content/drive/MyDrive/HTML final project/Train_IDs.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/HTML final project/sample_submission.csv')

# change string to value
status['Churn Category'].replace(['No Churn', 'Competitor', 'Dissatisfaction', 'Attitude', 'Price', 'Other'], [0, 1, 2, 3, 4, 5], inplace = True)

# merge all dataframe

In [ ]:
# merge all dataframe and assign it to "data"
# will lose a few datapoints (rows) because there are some different "Customer ID" in different csv
data = pd.merge(pd.merge(pd.merge(demographics, location, on=["Customer ID", "Count"], how="outer"),
                          pd.merge(services, satisfaction, on="Customer ID", how="outer"),
                          on=["Customer ID", "Count"] , how="outer"),
                status, on="Customer ID" ,)

In [ ]:
data.shape

(5110, 46)

In [ ]:
data.to_csv('/content/drive/MyDrive/HTML final project/new_data.csv')

# anaylsis

- 代表 sat score < 3，就不用考慮 No churn
- 但 sat score > 3，不一定是 No churn

In [ ]:
status = ['No Churn', 'Competitor', 'Dissatisfaction', 'Attitude', 'Price', 'Other']
for i in range(6):
  print()
  print(status[i])
  print('sat score:')
  print('mean: ', data[data['Churn Category'] == i]['Satisfaction Score'].mean())
  print('median: ', data[data['Churn Category'] == i]['Satisfaction Score'].median())
  print()
  print('sat score <  3:', (data[data['Churn Category'] == i]['Satisfaction Score'] < 3).sum())
  print('sat score >= 3:', (data[data['Churn Category'] == i]['Satisfaction Score'] >= 3).sum())
  print('-----------------')


No Churn
sat score:
mean:  3.786476868327402
median:  4.0

sat score <  3: 0
sat score >= 3: 1405
-----------------

Competitor
sat score:
mean:  1.613953488372093
median:  1.0

sat score <  3: 171
sat score >= 3: 44
-----------------

Dissatisfaction
sat score:
mean:  1.7875
median:  2.0

sat score <  3: 59
sat score >= 3: 21
-----------------

Attitude
sat score:
mean:  1.9901960784313726
median:  2.0

sat score <  3: 74
sat score >= 3: 28
-----------------

Price
sat score:
mean:  1.7959183673469388
median:  2.0

sat score <  3: 37
sat score >= 3: 12
-----------------

Other
sat score:
mean:  2.122448979591837
median:  2.0

sat score <  3: 33
sat score >= 3: 16
-----------------


- sat score = 4 or 5 一定是 0
- sat score = 3 最混亂

In [ ]:
for i in range(6):
  print()
  print(i)
  print(data[data['Churn Category'] == i]['Satisfaction Score'].value_counts()) # competitor at city 849 and 314 550


0
3.0    606
4.0    493
5.0    306
Name: Satisfaction Score, dtype: int64

1
1.0    127
3.0     44
2.0     44
Name: Satisfaction Score, dtype: int64

2
1.0    38
3.0    21
2.0    21
Name: Satisfaction Score, dtype: int64

3
2.0    45
1.0    29
3.0    28
Name: Satisfaction Score, dtype: int64

4
1.0    22
2.0    15
3.0    12
Name: Satisfaction Score, dtype: int64

5
2.0    23
3.0    16
1.0    10
Name: Satisfaction Score, dtype: int64


# 'Churn Category' == 1 (用 city 解決)
- sat score < 3 且 住在 city 849, 314 的 Churn Category 都是 1 (550 不一定)

In [ ]:
print('Churn Category == 1 count: ', data[data['Satisfaction Score'] == 1].shape[0])
print()
print(data[data['City'] == 849]['Churn Category'].value_counts())
print(data[data['City'] == 550]['Churn Category'].value_counts())
print(data[data['City'] == 314]['Churn Category'].value_counts())
print(data[data['City'] == 853]['Churn Category'].value_counts())
print(data[data['City'] == 982]['Churn Category'].value_counts())
print('\nChurn Category == 1 Contract: ')
print(data[data['Churn Category'] == 1]['Contract'].value_counts())

print('\n排除後：')
print(data[(data['City'] != 849) & (data['City'] != 314) & (data['Churn Category'] == 1)].shape[0])
# print('\nChurn Category == 1 city: ')
# data[data['Churn Category'] == 1]['City'].value_counts() # competitor at city 849 and 314 550

Churn Category == 1 count:  226

1    48
0    37
5     2
3     1
Name: Churn Category, dtype: int64
0    61
1    10
2     5
5     3
3     2
4     1
Name: Churn Category, dtype: int64
1    7
0    3
5    1
Name: Churn Category, dtype: int64
0    2
Name: Churn Category, dtype: int64
0    6
1    5
3    1
Name: Churn Category, dtype: int64

Churn Category == 1 Contract: 
0.0    185
1.0     18
2.0      8
Name: Contract, dtype: int64

排除後：
196


# 'Churn Category' == 2
- City: (79 / 1182)

In [ ]:
print(data[data['Churn Category'] == 2]['Latitude'].value_counts())
print()
print(len(data['Latitude'].unique()))

38.526941    2
34.079934    2
37.311088    1
34.213049    1
33.695532    1
            ..
37.590421    1
37.089111    1
38.554383    1
38.432145    1
37.629840    1
Name: Latitude, Length: 91, dtype: int64

1276


# 'Churn Category' == 3
sat score < 3 且 Contract == 0 不是 1 就是 3

In [ ]:
print(data[data['Churn Category'] == 3]['Contract'].value_counts())
print()
print('Contract == 0 distributuion:')
print(data[data['Contract'] == 0]['Churn Category'].value_counts())

0.0    101
1.0      6
2.0      1
Name: Contract, dtype: int64

Contract == 0 distributuion:
0    543
1    185
3    101
2     68
4     42
5     39
Name: Churn Category, dtype: int64


In [ ]:
print(data[data['Churn Category'] == 4]['City'].value_counts().shape[0])
print()
print(len(data['City'].unique()))

55

872


,Customer ID,Count,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Country,State,City,Zip Code,Latitude,Longitude,Population,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Churn Category
0,1746-TGTWV,1.0,1.0,78.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,550.0,90022.0,34.023810,-118.156582,68701.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00,0.0,1.0,1.0,8.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,39.65,NaN,0.00,20.0,0.00,59.65,NaN,1
1,5832-EXGTT,1.0,0.0,78.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,445.0,90303.0,33.936291,-118.332639,27778.0,0.0,1.0,1.0,25.0,3.0,NaN,19.76,0.0,1.0,NaN,12.0,0.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,0.0,1.0,0.0,98.50,2514.50,NaN,0.0,494.00,2995.07,2.0,2
2,3458-IDMFK,1.0,0.0,76.0,0.0,1.0,1.0,NaN,NaN,0.0,0.0,548.0,90720.0,33.794990,-118.065591,21343.0,0.0,1.0,1.0,1.0,5.0,1.0,NaN,NaN,1.0,2.0,30.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,70.45,70.45,0.00,0.0,15.28,85.73,2.0,5
3,3196-NVXLZ,1.0,1.0,65.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,34.097345,-117.906736,NaN,0.0,0.0,0.0,NaN,4.0,1.0,29.11,0.0,1.0,2.0,29.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1.0,NaN,NaN,1074.30,NaN,0.0,436.65,1510.95,1.0,1
4,3522-CDKHF,1.0,1.0,77.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,849.0,92122.0,32.857230,-117.209774,34902.0,0.0,1.0,1.0,24.0,3.0,1.0,16.10,0.0,1.0,NaN,12.0,0.0,0.0,1.0,0.0,1.0,NaN,1.0,1.0,0.0,1.0,0.0,93.15,2231.05,0.00,0.0,386.40,2617.45,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,9114-DPSIA,1.0,1.0,53.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,757.0,92268.0,34.201108,-116.593456,354.0,0.0,1.0,2.0,NaN,2.0,1.0,21.39,1.0,1.0,2.0,10.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,1.0,NaN,1.0,1.0,84.80,3626.35,43.96,NaN,NaN,4523.55,4.0,0
2168,3199-NPKCN,1.0,0.0,51.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,794.0,92270.0,33.763678,-116.429928,12465.0,0.0,0.0,0.0,18.0,NaN,1.0,7.42,1.0,1.0,2.0,12.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,NaN,1679.40,38.65,0.0,133.56,1774.31,4.0,0
2169,3329-WDIOK,1.0,1.0,63.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,1061.0,NaN,33.036790,-115.605030,NaN,0.0,0.0,0.0,38.0,0.0,1.0,NaN,0.0,1.0,2.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,1.0,0.0,1.0,1.0,69.50,2625.25,20.19,0.0,1331.52,3936.58,3.0,0
2170,4835-YSJMR,1.0,1.0,38.0,0.0,0.0,NaN,1.0,2.0,0.0,0.0,2.0,92301.0,34.667815,-117.536183,18980.0,0.0,1.0,1.0,24.0,NaN,1.0,36.05,1.0,1.0,0.0,24.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,84.80,1990.50,48.23,0.0,865.20,2807.47,3.0,0


文忠嘗試施工區


SyntaxError: ignored